This is for Capstone Project on Coursera.  author: yang cao.

In [180]:
import pandas as pd
import numpy as np

In [181]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### Week 3 Assignment - Q1
Getting data from Wikipedia using BeautifulSoup, and generate the dataframe.

In [182]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import os
import pandas as pd

In [183]:
#request content from url.
http = httplib2.Http()
status, response = http.request('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

#make soup.
soup = BeautifulSoup(response, 'html.parser')

#extract data from table.
table = soup.find('table', {'class': 'wikitable sortable'})
post_code = []
borough =[]
neighborhood = []
for td in table.find_all('tr'):
    pc, bg, nb = td.get_text().splitlines()[1:] 
    
    if bg != "Not assigned":      #ignore borough of not assigned.
        if nb == "Not assigned":  #if neighborhood not assigned, assign it as borough.
            nb = bg     
        post_code.append(pc)
        borough.append(bg)
        neighborhood.append(nb)
        
#store table data as dataframe.        
df = pd.DataFrame(list(zip(post_code,borough,neighborhood)))  
df.columns = df.iloc[0]                
df = df[1:]

#put neighborhoods with the same "post_code" and "Borough" together.
g=df.groupby(['Postcode','Borough'])
post_code=[]
borough=[]
neighborhood=[]
for key, value in g.groups.items(): #g.groups is a dictionary.
    post_code.append(key[0])        
    borough.append(key[1])
    temp = df.loc[value]
    neighborhood.append(temp['Neighbourhood'].str.cat(sep=','))


df1 = pd.DataFrame(list(zip(post_code,borough,neighborhood)))
df1.columns=df.columns  #df1 is the final dataframe.

__Print out the dataframe:__

In [184]:
#print output the top 5 rows.
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


__The shape of the dataframe is__

In [185]:
print('The shape of the dataframe is: ', df1.shape)
#df1.to_csv('data_updated.csv', index='False')

The shape of the dataframe is:  (103, 3)


### Week 3 Assignment - Q2
get the latitude and the longitude coordinates of each neighborhood.

In [186]:
#download the geospatial data.
Geo_data = pd.read_csv('http://cocl.us/Geospatial_data')

In [187]:
Geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [188]:
Geo_data=Geo_data.rename(index=str, columns={"Postal Code": "Postcode"})
Geo_data['Postcode']=Geo_data.Postcode.astype(str)

__adding latitude & longitude, the dataframe becomes:__

In [189]:
df1=df1.merge(Geo_data, on='Postcode',how='left')
df1.head()
#df1.to_csv('data_updated2.csv', index='False')

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Week 3 Assignment - Q3
Explore and cluster the neighborhoods in Toronto.
__Neighborhood: Scarborough is studied__

#### Create a map of Toronto with neighborhoods superimposed on top.

In [190]:
import folium # map rendering library

lat_toronto = 43.6532
lon_tornoto = -79.3832

map_toronto = folium.Map(location=[lat_toronto, lon_tornoto], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

#### select neighbourhood scarborough for study

In [191]:
scarborough_data = df1[df1['Borough']=='Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [192]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [193]:
# create map of Manhattan using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

#### explore and segment the neighbourhoods using Foursquare API. 

In [194]:
CLIENT_ID = '3WNK3M0ZLXUIZKK5HGQGNOWJGNOYYJRVSOZ2DXWICIL3EECH' # your Foursquare ID
CLIENT_SECRET = 'A550XJTHWQQ5EF24ICUQFCURJQKNGE4F3T0DCTC2XEOXCKP0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3WNK3M0ZLXUIZKK5HGQGNOWJGNOYYJRVSOZ2DXWICIL3EECH
CLIENT_SECRET:A550XJTHWQQ5EF24ICUQFCURJQKNGE4F3T0DCTC2XEOXCKP0


Let us explore the neighbourhoods corresponding to the first postcode in scarborough_data.

In [195]:
scarborough_data.loc[0,'Neighbourhood']

'Rouge,Malvern'

In [196]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


#### search the top 100 venues that in Rouge and Malvern within a radius of 500 meters.

In [197]:
search_query = 'venues'
radius = 500
LIMIT = 100
# type your answer here
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=3WNK3M0ZLXUIZKK5HGQGNOWJGNOYYJRVSOZ2DXWICIL3EECH&client_secret=A550XJTHWQQ5EF24ICUQFCURJQKNGE4F3T0DCTC2XEOXCKP0&ll=43.806686299999996,-79.19435340000001&v=20180605&radius=500&limit=100'

In [198]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

results = requests.get(url).json()
results
with open('results.json', 'w') as fp:
    json.dump(results, fp)

Borrow the __get_category_type__ function from the Foursquare lab.

In [199]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe.

In [200]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [201]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


#### Explore neighbourhoods in Scarborough.

In [202]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called scarborgouh_venues.

In [203]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )


Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge


check the size of the resulting dataframe.

In [204]:
print(scarborough_venues.shape)
scarborough_venues.head()

(92, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [205]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,6,6,6,6,6,6
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Birch Cliff,Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea,Golden Mile,Oakridge",10,10,10,10,10,10
"Clarks Corners,Sullivan,Tam O'Shanter",11,11,11,11,11,11
"Cliffcrest,Cliffside,Scarborough Village West",3,3,3,3,3,3
"Dorset Park,Scarborough Town Centre,Wexford Heights",7,7,7,7,7,7
"East Birchmount Park,Ionview,Kennedy Park",4,4,4,4,4,4


find out how many unique categories can be curated from all the returned venues.

In [206]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 54 uniques categories.


#### Analyze each neighborhood.

In [207]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [208]:
scarborough_onehot.shape

(92, 55)

group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [209]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.0,0.000000,0.000,0.0,0.166667,0.0,0.0,...,0.166667,0.0,0.166667,0.0,0.166667,0.0,0.0,0.000,0.0,0.0
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000,0.0,0.333333,0.000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0
2,"Birch Cliff,Cliffside West",0.0,0.000,0.0,0.000000,0.000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.250000,0.0,0.0,0.000,0.0,0.0
3,Cedarbrae,0.0,0.125,0.0,0.125000,0.125,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.125,0.0,0.0
4,"Clairlea,Golden Mile,Oakridge",0.0,0.000,0.0,0.200000,0.000,0.0,0.000000,0.2,0.1,...,0.000000,0.0,0.000000,0.0,0.000000,0.1,0.0,0.000,0.0,0.0


Let's print each neighborhood along with the top 5 most common venues

In [210]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0              Lounge  0.17
1      Breakfast Spot  0.17
2        Skating Rink  0.17
3      Sandwich Place  0.17
4  Chinese Restaurant  0.17


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                 venue  freq
0               Bakery  0.33
1           Playground  0.33
2                 Park  0.33
3  American Restaurant  0.00
4            Pet Store  0.00


----Birch Cliff,Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                venue  freq
0              Lounge  0.12
1              Bakery  0.12
2                Bank  0.12
3     Thai Restaurant  0.12
4  Athletics & Sports  0.12


----Clairlea,Golden Mile,Oakridge----
           venue  freq
0         Bakery   0.2
1       Bus Line   0.2
2  Metro Station   0.1
3   Intersection   0.1
4   Soccer

write a function to sort the venues in descending order.

In [211]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create the new dataframe and display the top 10 venues for each neighborhood.

In [212]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Chinese Restaurant,Skating Rink,Sandwich Place,Print Shop,Convenience Store,Department Store,Discount Store,Electronics Store
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Bakery,Playground,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,"Birch Cliff,Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Grocery Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Lounge,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
4,"Clairlea,Golden Mile,Oakridge",Bus Line,Bakery,Intersection,Fast Food Restaurant,Metro Station,Bus Station,Park,Soccer Field,Bar,Convenience Store


#### cluster neighborhoods.

In [213]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([1, 1, 1, 1, 1, 1, 4, 1, 3, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [214]:
#add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

__since Upper Rouge has no nearby venues (outputed from getNearbyVenues), delete its data from scarborough_data using scarborough_data.drop(16)__ 

In [215]:
scarborough_merged = scarborough_data.drop(16)
scarborough_merged = scarborough_merged.rename(index = str, columns ={'Neighbourhood':'Neighborhood'})

#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,1,Bar,Moving Target,Construction & Landscaping,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1,Intersection,Spa,Electronics Store,Rental Car Location,Breakfast Spot,Medical Center,Pizza Place,Mexican Restaurant,College Stadium,Furniture / Home Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Korean Restaurant,Convenience Store,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Lounge,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store


let's visualize the resulting clusters

In [216]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine clusters.

In [217]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store


In [218]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1,Bar,Moving Target,Construction & Landscaping,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
2,Scarborough,1,Intersection,Spa,Electronics Store,Rental Car Location,Breakfast Spot,Medical Center,Pizza Place,Mexican Restaurant,College Stadium,Furniture / Home Store
4,Scarborough,1,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Lounge,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
7,Scarborough,1,Bus Line,Bakery,Intersection,Fast Food Restaurant,Metro Station,Bus Station,Park,Soccer Field,Bar,Convenience Store
9,Scarborough,1,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Grocery Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
10,Scarborough,1,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Furniture / Home Store,Chinese Restaurant,Pet Store,Construction & Landscaping,Grocery Store,General Entertainment,Fried Chicken Joint
11,Scarborough,1,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Furniture / Home Store
12,Scarborough,1,Lounge,Breakfast Spot,Chinese Restaurant,Skating Rink,Sandwich Place,Print Shop,Convenience Store,Department Store,Discount Store,Electronics Store
13,Scarborough,1,Pizza Place,Noodle House,Shopping Mall,Pharmacy,Fast Food Restaurant,Bank,Chinese Restaurant,Italian Restaurant,Fried Chicken Joint,Thai Restaurant
14,Scarborough,1,Park,Bakery,Playground,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [219]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2,Playground,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [220]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,3,Coffee Shop,Korean Restaurant,Convenience Store,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,Scarborough,3,Coffee Shop,Discount Store,Department Store,Playground,Vietnamese Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


In [221]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,4,Motel,American Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
